# JSON Exercise Solutions

+ Andy Pickering
+ May 23, 2017

## Summary

This notebook contains my solutions to the JSON data wrangling exercises, as part of the Springboard data science curriculum.

****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
# import libraries
import pandas as pd
import json
from pandas.io.json import json_normalize

****
## JSON exercise questions

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

# (1) Find the 10 countries with most projects

In [2]:
# read json file into a dataframe
df = pd.read_json('data/world_bank_projects.json')
df.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


To find the countries with the most projects, I group by 'countryname', compute *size()* and sort in descending order.

In [3]:
# group by country and count # rows (since each row is a project)
df.groupby('countryname').size().sort_values(ascending=False).head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
dtype: int64

I could also use *value_counts()* :

In [4]:
df.countryname.value_counts().head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Nepal                              12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

One thing to note is that Africa is listed as a country, instead of the specific countries in Africa. If we wanted to get the specific countries, it looks like we might be able to get some information from the 'borrower' column? However I won't go any further here.

In [5]:
df[['borrower','countryname']][df.countryshortname=='Africa'].head()

,borrower,countryname
45,ECOWAS,Africa
46,UGANDA-COMOROS,Africa
51,"OSS, IUCN, CILSS",Africa
58,BANK EXECUTED,Africa
65,"BURUNDI,RWANDA,TANZANIA",Africa


# (2) Find the top 10 major project themes (using column 'mjtheme_namecode')

Using the dataframe as read in in (1) has some issues; there are multiple codes/themes for the same project, and the name is missing for some of them.

In [6]:
# example:
df['mjtheme_namecode'][0]

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]

But we can create a separate data frame with all the codes and names from 'mjtheme_namecode', using *json_normalize* .

In [7]:
a=json.load((open('data/world_bank_projects.json')))
b = json_normalize(a,'mjtheme_namecode',['id','countryname'])
b.head(10)

,code,name,id,countryname
0,8,Human development,P129828,Federal Democratic Republic of Ethiopia
1,11,,P129828,Federal Democratic Republic of Ethiopia
2,1,Economic management,P144674,Republic of Tunisia
3,6,Social protection and risk management,P144674,Republic of Tunisia
4,5,Trade and integration,P145310,Tuvalu
5,2,Public sector governance,P145310,Tuvalu
6,11,Environment and natural resources management,P145310,Tuvalu
7,6,Social protection and risk management,P145310,Tuvalu
8,7,Social dev/gender/inclusion,P144665,Republic of Yemen
9,7,Social dev/gender/inclusion,P144665,Republic of Yemen


To answer question (2), i'll here use *value_counts()* to find the most frequent project codes. After question (3) i'll produce the same list of project *names*, since question (3) involves filling in the missing names.

In [8]:
b.code.value_counts().head(10)

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64

See end of (3) for same result, with project *names*.

# (3) In 2. above you will notice that some entries have only the code and the name is missing.  Create a dataframe with the missing names filled in.

In [9]:
# print df w/ missing names
b.head(20)

,code,name,id,countryname
0,8,Human development,P129828,Federal Democratic Republic of Ethiopia
1,11,,P129828,Federal Democratic Republic of Ethiopia
2,1,Economic management,P144674,Republic of Tunisia
3,6,Social protection and risk management,P144674,Republic of Tunisia
4,5,Trade and integration,P145310,Tuvalu
5,2,Public sector governance,P145310,Tuvalu
6,11,Environment and natural resources management,P145310,Tuvalu
7,6,Social protection and risk management,P145310,Tuvalu
8,7,Social dev/gender/inclusion,P144665,Republic of Yemen
9,7,Social dev/gender/inclusion,P144665,Republic of Yemen


In this case there are only 11 unique codes/project names, so I could just manually code the names and use a loop to fill them in. But i'd like to do it in a more automatic way that would work if we had many more codes. 

I thought about sorting by code and then using *ffill* to fill in/replace the missing names. This would work for most cases. But note what happens what if we happen to have empty name for the first row of a new code: In the example below, the first row of code 5 is incorrectly filled in with the name for code 8.

In [10]:
ex_df = pd.DataFrame({'code':[8,8, 8, 5, 5] , 'name': ['Human Development','','','','Trade and Integration']})
ex_df.head()

,code,name
0,8,Human Development
1,8,
2,8,
3,5,
4,5,Trade and Integration


In [11]:
ex_df.name = ex_df.name.replace('',method='ffill')
ex_df.head()

,code,name
0,8,Human Development
1,8,Human Development
2,8,Human Development
3,5,Human Development
4,5,Trade and Integration


Instead, I just get the rows for each unique code, find the first non-empty name, and fill in all rows with the name.

In [12]:
# loop over unique codes, get name, and fill in all rows with names
for code in b.code.unique():
    code_rows = b.name[b.code== code] # find rows w/ this code
    code_name = code_rows[code_rows!=''].values[1]   # find first non-empty name
    b.name[b.code==code] = code_name    # fill all rows of this code w/ name
    
# print df w/ names filled in
b.head(20)

,code,name,id,countryname
0,8,Human development,P129828,Federal Democratic Republic of Ethiopia
1,11,Environment and natural resources management,P129828,Federal Democratic Republic of Ethiopia
2,1,Economic management,P144674,Republic of Tunisia
3,6,Social protection and risk management,P144674,Republic of Tunisia
4,5,Trade and integration,P145310,Tuvalu
5,2,Public sector governance,P145310,Tuvalu
6,11,Environment and natural resources management,P145310,Tuvalu
7,6,Social protection and risk management,P145310,Tuvalu
8,7,Social dev/gender/inclusion,P144665,Republic of Yemen
9,7,Social dev/gender/inclusion,P144665,Republic of Yemen


And now that all rows have names, we can make the solution to (2), using the project *names* instead of the code numbers.

In [13]:
# top project names
b.groupby('name').size().sort_values(ascending=False).head(10)

name
Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
dtype: int64